In [1]:
import pandas as pd
import re
import numpy as np

from results_functions_general import reformat, joinkey

In [35]:
### CHANGE ###
pres_2024_filepath = 'source/presidential_11.6.10am.csv' 

In [36]:
df = pd.read_csv(pres_2024_filepath, skiprows=2, names=('precinct', 'pres_voters_2024','harris', 'harris_pct',
             'trump_2024', 'trump_2024_pct', 'kennedy', 'kennedy_pct'))

In [37]:
df.head()

,precinct,pres_voters_2024,harris,harris_pct,trump_2024,trump_2024_pct,kennedy,kennedy_pct
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ward 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Precinct,Total Voters,Kamala D. Harris & Tim Walz,%,Donald J. Trump & JD Vance,%,Robert F. Kennedy Jr. & Nicole Shanahan,%
3,1,942,830,0.00%,95,0.00%,12,100.00%
4,2,961,857,0.00%,96,0.00%,8,0.00%


In [38]:
# format
df = reformat(df)
df = joinkey(df)

In [39]:
df['pres_voters_2024'] = df['pres_voters_2024'].astype(int)
df['trump_2024'] = df['trump_2024'].astype(int)
df['harris'] = df['harris'].astype(int)  

# calculate float candidate percentages
df['trump_2024_pct'] = df['trump_2024']/df['pres_voters_2024']
df['harris_pct'] = df['harris']/df['pres_voters_2024']

In [40]:
# calculate winners

# add winner column
df['2024_winner'] = '' 

for index, row in df.iterrows():
    if row['trump_2024_pct'] > 0.5:
        df.at[index, '2024_winner'] = 'Trump'
    elif row['harris_pct'] > 0.5:
        df.at[index, '2024_winner'] = 'Harris'
    else:
        df.at[index, '2024_winner'] = 'Not Reported'

In [30]:
df.head(10)

,precinct,pres_voters_2024,harris,harris_pct,trump_2024,trump_2024_pct,kennedy,kennedy_pct,ward,ward_precinct,2024_winner
3,1,942,830,0.881104,95,0.100849,12,100.00%,1,1_1,Harris
4,2,961,857,0.891779,96,0.099896,8,0.00%,1,1_2,Harris
5,3,943,835,0.885472,94,0.099682,12,100.00%,1,1_3,Harris
6,4,1027,937,0.912366,84,0.081792,6,0.00%,1,1_4,Harris
7,5,999,891,0.891892,95,0.095095,12,100.00%,1,1_5,Harris
8,6,847,730,0.861865,106,0.125148,11,0.00%,1,1_6,Harris
9,7,978,862,0.881391,105,0.107362,10,100.00%,1,1_7,Harris
10,8,920,783,0.851087,124,0.134783,12,0.00%,1,1_8,Harris
11,9,589,461,0.782683,121,0.205433,7,100.00%,1,1_9,Harris
12,10,1167,1024,0.877464,132,0.113111,8,0.00%,1,1_10,Harris


In [41]:
# export
df.to_csv('processed/presidential_11.6_10am.csv')

## Analysis

In [32]:
# how many precincts did trump win?
len(df)

1291

In [33]:
len(df[df['2024_winner'] == 'Trump'])

101

In [34]:
(len(df[df['2024_winner'] == 'Trump'])/len(df))*100

7.8233927188226176

In [42]:
# load turnout
turnout = pd.read_csv('processed/chi_turnout_11.6_830am.csv')

In [44]:
turnout.tail()

,Unnamed: 0,precinct,registered_voters,ballots_cast,turnout,ward,ward_precinct,turnout_formatted,Unnamed: 8
1286,1485,22,1803,1500,0.831947,42,42_22,83.19%,NaN
1287,1486,6,1868,1511,0.808887,43,43_6,80.89%,NaN
1288,1487,17,1818,1556,0.855886,44,44_17,85.59%,NaN
1289,1488,9,1937,1561,0.805885,43,43_9,80.59%,NaN
1290,1489,16,2037,1589,0.780069,27,27_16,78.01%,NaN


In [45]:
# merge turnout
df = df.merge(turnout[['registered_voters', 'ward_precinct','ballots_cast']], on='ward_precinct')

In [47]:
# 2024 turnout by candidate
g = df.groupby('2024_winner')[['ballots_cast','registered_voters']].sum()
g['turnout'] = g['ballots_cast']/g['registered_voters']
g

,ballots_cast,registered_voters,turnout
2024_winner,,,
Harris,795744,1334155,0.596440
Not Reported,10410,63273,0.164525
Trump,66209,101445,0.652659


In [2]:
# load comparisons
hist = pd.read_csv('comparisons/2020_and_2016_presidentials.csv')

In [3]:
hist.columns

Index(['Unnamed: 0', 'precinct', 'registered_voters_2016', 'ballots_cast_2016',
       'turnout_2016', 'ward', 'ward_precinct', 'registered_voters_2020',
       'ballots_cast_2020', 'turnout_2020', 'turnout_2020_formatted',
       'turnout_2016_formatted', 'pres_voters_2016', 'trump_2016',
       'trump_2016_pct', 'clinton', 'clinton_pct', 'pres_voters_2020',
       'trump_2020', 'trump_2020_pct', 'biden', 'biden_pct', '2016_winner',
       '2020_winner'],
      dtype='object')